# Getting the most out of GeoPandas 1.0

After 10 years since the first release, GeoPandas reached version 1.0. This workshop will showcase how to get the most out of the recent enhancements and develop a code ready for 2024 and beyond.

**Martin Fleischmann, Joris van den Bossche**

27/05/2024, Basel

## Setup

Follow the ReadMe to set up the environment correctly. You should have these packages installed:

```
- geopandas 1.0dev
- pyarrow
- geodatasets
- matplotlib
- folium
- mapclassify
```

## What is GeoPandas?

**Easy, fast and scalable geospatial analysis in Python**

From the docs:

> The goal of GeoPandas is to make working with geospatial data in python easier. It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS.

In [ ]:
import geopandas
import pandas
import numpy
from matplotlib.colors import ListedColormap

from geodatasets import get_path

## Union and the power of coverage

GeoPandas had historically a `unary_union` property to get a union of all geometries into a single one. Since 1.0, the property is deprecated in favour of a new `union_all` method, that offers additional `method` keyword.

See how it works on the example of NYC data.

In [ ]:
nyc = geopandas.read_file(get_path("geoda nyc education"), columns=["BoroCode"])
nyc

In [ ]:
nyc.explore("BoroCode", tiles="Carto DB Positron")

Doing union of all geometries is a simple call of the `union_all()` method. However, if you know that the geometries form a coverage, you can instruct GeoPandas to use optimized coverage union algorithm under the hood.

In [ ]:
%timeit nyc.union_all()
%timeit nyc.union_all(method="coverage")

Same option is exposed in the `dissolve` method, doing union within a group.

In [ ]:
boroughs = nyc.dissolve("BoroCode")
boroughs

In [ ]:
boroughs.explore(tiles="CartoDB Positron")

In this specific case, both options return the same result but at a different performance cost. 

In [ ]:
%timeit nyc.dissolve("BoroCode")
%timeit nyc.dissolve("BoroCode", method="coverage")

If you know that you are dealing with coverages, remember to specify the `method="coverage"`.

## Dot density mapping using `sample_points()`

Dot density mapping is a popular method of visualising data spread across multiple variables, like population counts per demographic group. While GeoPandas does not aim to provide advanced visualisation tooling, its new `sample_points()` method provides necessary tools to create a dot density map by hand.

Load the data representing Chicago, that contains the population counts for 2014.

In [ ]:
chicago = geopandas.read_file(get_path("geoda chicago_health"), columns=["Hisp14", "Blk14", "AS14", "Wht14", "TRACTCnt"])
chicago.head()

The most basic dot density map is reflecting a single variable. In our case, population of people identified as white.

You need to sample random points within each area. Given the large population, sample points so that one point represent 100 people.

In [ ]:
white = chicago.sample_points(chicago.Wht14 // 100, rng=42)
white.head()

The result can be directly plotted to a map, showing the density of white population across the city.

In [ ]:
ax = white.plot(color="k", markersize=0.01, figsize=(8, 8))
chicago.boundary.plot(ax=ax, color="k", linewidth=0.2)
ax.set_axis_off()

The true power of dot density mapping comes with mutiple layers. Sample points for the other demographic groups.

In [ ]:
hispanic = chicago.sample_points(chicago.Hisp14 // 100, rng=42)
black = chicago.sample_points(chicago.Blk14 // 100, rng=42)
asian = chicago.sample_points(chicago.AS14 // 100, rng=42)

With four `GeoSeries`, you can create a map manualy.

In [ ]:
ax = white.plot(color="#F2CF63", markersize=0.01, figsize=(8, 8))
hispanic.plot(color="#ADD9C9", markersize=0.01, ax=ax)
black.plot(color="#D96459", markersize=0.01, ax=ax)
asian.plot(color="#3B2F40", markersize=0.01, ax=ax)
ax.set_axis_off()

Though it may be more useful, to turn it all into a single GeoDataFrame. For visualisation purposes, unioning all geometries within a group will work nicely.

In [ ]:
dot_density = geopandas.GeoDataFrame(
    {"population": ["white", "hispanic", "black", "asian"]},
    geometry=[
        white.union_all(),
        hispanic.union_all(),
        black.union_all(),
        asian.union_all(),
    ],
    crs=chicago.crs,
)
dot_density

Now, you can use a single call to `plot()` to get a map, including a legend.

In [ ]:
cmap = ListedColormap(["#3B2F40", "#D96459", "#ADD9C9", "#F2CF63"])
ax = dot_density.plot(
    column="population",
    cmap=cmap,
    legend=True,
    markersize=0.01,
    figsize=(8, 8),
    legend_kwds=dict(loc="upper right", bbox_to_anchor=(1, 0.95), frameon=False),
)
chicago.boundary.plot(ax=ax, color="k", linewidth=0.1)
ax.set_axis_off()

The same also works with the interactive `explore()` method.

In [ ]:
m = chicago.boundary.explore(
    tiles="Carto DB Positron",
    prefer_canvas=True,
    color="black",
    )
dot_density.explore(
    column="population",
    cmap=cmap,
    legend=True,
    marker_kwds=dict(radius=1),
    m=m
)

## Triangulation

GeoPandas 1.0 brings methods for triangulation using the vertices of geometries. See the example of grocery locations in Chicago.

In [ ]:
groceries = geopandas.read_file(get_path("geoda groceries"))
groceries.head()

In [ ]:
groceries.explore(tiles="CartoDB Positron")

### Voronoi polygons

One new method allows you to generate Voronoi polygons around points.

In [ ]:
voronoi = groceries.voronoi_polygons()
voronoi

In [ ]:
m = voronoi.explore(tiles="CartoDB Positron")
groceries.explore(m=m, color="red")

Optionally, you can request just the edges of the tessellation as LineStrings.

In [ ]:
voronoi_edges = groceries.voronoi_polygons(only_edges=True)
voronoi_edges

In [ ]:
m = voronoi_edges.explore(tiles="CartoDB Positron")
groceries.explore(m=m, color="red")

### Delaunay

The inverse of Voronoi is Delaunay triangulation, creating polygons between the vertices.

In [ ]:
delaunay = groceries.delaunay_triangles()
delaunay

In [ ]:
m = delaunay.explore(tiles="CartoDB Positron")
groceries.explore(m=m, color="red")

Again, you can optionally work only with the edges.

In [ ]:
delaunay_edges = groceries.delaunay_triangles(only_edges=True)
delaunay_edges

In [ ]:
m = delaunay_edges.explore(tiles="CartoDB Positron")
groceries.explore(m=m, color="red")

The same algorithms work not only on points, but also on any other geometry type, by extracting the vertices from individual geometries.

In [ ]:
chicago.head()

In [ ]:
chicago_delaunay = chicago.delaunay_triangles(only_edges=True)

m = chicago_delaunay.explore(tiles="CartoDB Positron", prefer_canvas=True)
chicago.boundary.explore(m=m, color="red")

TIP: If you want to generate Voronoi tessellation that treats LineStrings and Polygons as a whole, not returing polygons per each vertex, you can use the [`voronoi_frames`](https://pysal.org/libpysal/generated/libpysal.cg.voronoi_frames.html#libpysal.cg.voronoi_frames) function from the `libpysal.cg` module of libpysal package.

## Extract unique points

If you want to check the individual vertices, for example to understand the input for triangulation, you can now use the `extract_unique_points()` method.

In [ ]:
points = chicago.extract_unique_points()
points

In [ ]:
points.explore(tiles="CartoDB Positron", prefer_canvas=True)

The map above gives a nice explanation of that Delaunay triangulation you did above.

## Polygonize and build_area

The `polygonize()` method is a new method that allows you to convert an array of LineStrings into a Polygons. This is useful when you have a LineStrings that represents closed shapes. Like the Delaunay edges.

In [ ]:
delaunay_edges.head()

The method considers all geometries together when doing the polygonization.

In [ ]:
polygons = delaunay_edges.polygonize()
polygons.head()

In [ ]:
polygons.explore(tiles="CartoDB Positron")

Similar method is `build_area()`, that uses the LineStrings to to build a single area, while treating enclosed lines as holes.

In [ ]:
delaunay_edges.build_area().explore(tiles="CartoDB Positron")

Let's add some artifical holes to illustrate the situation.

In [ ]:
linework = pandas.concat(
    [voronoi_edges, groceries.buffer(1000).boundary], ignore_index=True
)

In [ ]:
linework.explore(tiles="CartoDB Positron")

Comparing the results of `polygonize()` and `build_area()` shows the difference in the output.

In [ ]:
linework.build_area().explore(tiles="CartoDB Positron")

In [ ]:
linework.polygonize().explore(tiles="CartoDB Positron")

## Segmentize

Sometimes, it may be useful to include additional points along the edges of geometries. This is useful when you want to densify the geometries for further processing. GeoPandas 1.0 has a new method `segmentize()` that allows you to do just that.

Check the vertices of the Delaunay triangulation done above.

In [ ]:
delaunay.extract_unique_points().explore(tiles="CartoDB Positron")

Using only the vertices, we have no clue about the original shape. Let's segmentize the edges to get a better understanding.

In [ ]:
denser = delaunay.segmentize(max_segment_length=1000)
denser.head()

In [ ]:
dense_points = denser.extract_unique_points()
dense_points.explore(tiles="CartoDB Positron")

Much better! Note that the distance between the points is no higher than the specified distance but not precisely the same. 

## Hilbert distance

For a long time, sorting based on geometries simply raised an error. This has recently changed with the introduction of the `hilbert_distance()` method. This method allows you to sort geometries based on their Hilbert distance, ensuring that there is a canonical way of sorting ensuring that the two geometries close to each other in the GeoDataFrame are also close in space.

Let's pick the points from the Delaunay triangulation and check how they are sorted.

In [ ]:
dense_points_df = dense_points.to_frame("geometry").explode()
dense_points_df

Generate a label for each quantile based on the order of rows.

In [ ]:
quantile_label = numpy.repeat(numpy.arange(100), len(dense_points_df) // 100 + 1)[:len(dense_points_df)]

In [ ]:
dense_points_df.explore(quantile_label, cmap="viridis", tiles="CartoDB Positron")

There seems to be some spatial order in the data but not necesarily very useful. Check how the extent of each quantile looks like.

In [ ]:
dense_points_df.dissolve(quantile_label).envelope.explore(tiles="CartoDB Positron")

Now let's do the same, but sort the GeoDataFrame based on geometry first.

In [ ]:
spatially_sorted = dense_points_df.sort_values("geometry")

The quantiles show the underlying Hilbert curve.

In [ ]:
spatially_sorted.explore(quantile_label, cmap="viridis", tiles="CartoDB Positron")

And the envelopes are much less overlapping.

In [ ]:
spatially_sorted.dissolve(quantile_label).envelope.explore(tiles="CartoDB Positron")

All of this is based on the `hilbert_distance()` method, which is called internally in the `sort_values()` method.

In [ ]:
spatially_sorted.hilbert_distance()

## Reading from sorted Parquet files

One application of spatially sorting is in reading subsets of large GeoParquet files. When only needing a part of the data, and the data is spatially sorted, we can make use of the chunked nature of Parquet files to automatically skip parts of the file.

By default, the geometries are serialized as WKB in the Parquet file. In that case, we can write an additional column with the bounding box of each feature.

Let's look at this Polygon dataset covering the US, and make it a bit bigger to see the difference:

In [ ]:
counties = geopandas.read_file(get_path("geoda.us_sdoh"))
counties = pandas.concat([counties]*10, ignore_index=True)

In [ ]:
len(counties)

Writing to Parquet and reading it back as a baseline:

In [ ]:
counties.to_parquet("counties.parquet", row_group_size=10000)

In [ ]:
%time _ = geopandas.read_parquet("counties.parquet")

Now we can write it with a bbox column using the new `write_covering_bbox=True` option, allowing us to read a subset of the data by specifying a bounding box:

In [ ]:
counties.to_parquet("counties_bbox.parquet", write_covering_bbox=True, row_group_size=10000)

In [ ]:
%time len(geopandas.read_parquet("counties_bbox.parquet"))

In [ ]:
bbox = list(counties[counties.county == "Bronx"].total_bounds)

In [ ]:
%time len(geopandas.read_parquet("counties_bbox.parquet", bbox=bbox))

Specifying a bounding box already speeds up the reading considerably. In this case it will generally not avoid loading all data from the Parquet file, but it will still avoid parsing all WKB values to geometries.

Let's now first sort the geometries before writing to Parquet, and then read the same subset again:

In [ ]:
counties_spatially_sorted = counties.sort_values("geometry").reset_index(drop=True)

In [ ]:
counties_spatially_sorted.to_parquet("counties_bbox_sorted.parquet", write_covering_bbox=True, row_group_size=10000)

In [ ]:
%time len(geopandas.read_parquet("counties_bbox_sorted.parquet", bbox=bbox))

In [ ]:
chicago_sorted = chicago.sort_values("geometry")
chicago_sorted["partition"] = numpy.repeat(numpy.arange(11), 7)
chicago_partitions = chicago_sorted.dissolve("partition")

The other can represent boundaries of community areas wihtin 2000 feet of the groceries.

In [ ]:
boundaries_near_shops = chicago_sorted.set_geometry(chicago_sorted.boundary).clip(groceries.buffer(2000).to_crs(chicago.crs))

Check both visually.

In [ ]:
m = chicago_partitions.explore(tiles="CartoDB Positron")
boundaries_near_shops.explore(m=m, color="red")

Now find the shared paths between the two sets of geometries. We need to properly align matching geometries as the method works on a row-wise basis.

In [ ]:
shared = chicago_partitions.boundary.shared_paths(boundaries_near_shops.set_index("partition"), align=True)
shared

The output is farily specific. The method returns a GeometryCollection with two components. The first one contains paths that have the same direction in both while the second one contains paths that have the opposite direction.

## `dwithin`, aka "distance within", joins and predicates

GeoPandas 1.0 brings a new method `dwithin()` that allows you to do spatial joins based on distance. This is useful when you want to join geometries that are within a certain distance from each other but want to avoid buffering,

Let's load the data on a subset of abandoned car locations in Chicago.

In [ ]:
cars = geopandas.read_file("data/cars.gpkg")
cars.head()

Ensure that the data is in the same CRS. Note that this shall be projected as youa re specifying the distance in units of the CRS.

In [ ]:
cars = cars.to_crs(groceries.crs)

Check the data.

In [ ]:
cars.explore(tiles="CartoDB Positron")

The distance join is done by using the `sjoin()` method with `predicate="dwithin"` and a set `distance`.

In [ ]:
cars_near_groceries = cars.sjoin(groceries, predicate="dwithin", distance=1000)
cars_near_groceries

The result is a subset of abandoned cars that are within 1000 feet of a groceries, with the grocery information attached.

In [ ]:
cars_near_groceries.explore("Chain", tiles="CartoDB Positron")

If you are not interested in the join, you can also use the predicate alongside the distance keyword in the `sindex.query()` method or use the `dwithin()` method.

## Force_2D and Force_3D

GeoPandas 1.0 brings two new methods to force geometries into 2D or 3D. This is useful when you want to ensure that all geometries have the same dimensionality and the input data is not consistent.

In [ ]:
chicago.head()

Check if there are some geometries that are not 2D.

In [ ]:
chicago.has_z.any()

Force all geometries to be 3D. This adds, by default, 0 as the z-coordinate.

In [ ]:
with_z = chicago.force_3d()
with_z.head()

And indeed, all geometries are now 3D.

In [ ]:
with_z.has_z.all()

But you can also specify the z-coordinate for each geometry individually.

In [ ]:
with_z_range = chicago.force_3d(z=range(len(chicago)))
with_z_range.head()

To do the opposite, force all geometries to be 2D, you can use the `force_2d()` method. This strips all z-coordinates.

In [ ]:
without_z = with_z_range.force_2d()
without_z.head()

## Hausdorff and Frechet distance

GeoPandas 1.0 brings two new methods to calculate distances between geometries. The Hausdorff distance and the Frechet distance. The standard `distance` method measures the shortest distance between two geometries. The `hausdorff_distance()` measures the longest distance between two geometries. The `frechet_distance()` measures the shortest distance between two points that traverse both geometries.

Let's illustrate it on a data representing large rivers in Europe.

In [ ]:
rivers = geopandas.read_file(get_path("eea large rivers")).set_index("NAME")
rivers

In [ ]:
rivers.explore(tiles="CartoDB Positron")

Frechet distance is often used to measure the similarity between two curves. Like between the original geometry and its simplified version.

In [ ]:
simplified_rivers = rivers.simplify(10_000)

Compare both visually.

In [ ]:
m = simplified_rivers.explore(tiles="CartoDB Positron")
rivers.explore(m=m, color='red')

And measure the distance.

In [ ]:
rivers.frechet_distance(simplified_rivers)

If you do that iteratively for different simplification levels, you will see how the distance changes.

In [ ]:
frechet = {}
for i in [1000, 2000, 5000, 10000]:
    simplified_rivers = rivers.simplify(i)
    frechet[i] = rivers.frechet_distance(simplified_rivers)

pandas.DataFrame(frechet).T.plot(legend=False)

Hausdorff distance is a bit different as it measures the longest distance between two geometries.

In [ ]:
rivers.hausdorff_distance(simplified_rivers)

You can see that the result is bounded by the tolerance of the simplification due to the underlying Douglas-Peucker algorithm.

In [ ]:
hausdorff = {}
for i in [1000, 2000, 5000, 10000]:
    simplified_rivers = rivers.simplify(i)
    hausdorff[i] = rivers.hausdorff_distance(simplified_rivers)

pandas.DataFrame(hausdorff).T.plot(legend=False)